# 1 数据 

## 读取 所有改变了的模型的数据




In [1]:
import os
import sys
sys.path.insert(0,os.path.abspath('/notebooks/pytorch_timeseries'))


In [2]:
## 1.2 读取 mean 数据


from dataclasses import asdict
import string
from openpyxl import Workbook
import pandas as pd
import wandb
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment

from torch_timeseries.experiments.dlinear_experiment import DLinearExperiment
# 初始化Wandb API
api = wandb.Api()

def convert_dict(dictionary):
    converted_dict = {}
    for key, value in dictionary.items():
        converted_dict[f"config.{key}"] = value
    return converted_dict

# 查询特定配置的运行
project_name = "BiSTGNN"  # 替换为你的项目名称
config_name = "your-config-name"  # 替换为你的配置名称

# single steps report

# models = ["GRU","DLinear", "NLinear", "TSMixer", "MICN","DeepTime","LaST", "Crossformer", "FiLM", "TNTCN_hetero_han"]
# datasets = ["ExchangeRate",  "ETTm1", "ETTm2", "ETTh1", "ETTh2"] #"ExchangeRate",
horizons = [3, 6, 12, 24]

all_data = {}
runs = api.runs(path="BiSTGNN")


/root/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.2 带 std的结果

In [81]:
# 获得带+-的结果
all_data = {}
for run in runs:
        if run.state == "finished":
            try:
                # if "TNTCN" in str(run.config['model_type']):
                if "TNTCN_hetero_han" in str(run.config['model_type']):
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mse')] = run.summary['mse']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2')] = run.summary['r2']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2w')] = run.summary['r2_weighted']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mae')] = run.summary['mae']
            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['dataset', 'horizon', 'metric', 'model_type'])
all_df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
result_df = all_df.unstack(0).loc[:,0]
result_df.to_excel('results.xlsx')

In [82]:
all_data = {}


# 获得带+-的结果
for run in runs:
        if run.state == "finished":
            try:
                if "TNTCN" in str(run.config['model_type']) or  "BiSTGNN" in str(run.config['model_type']):
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mse')] = run.summary['mse_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2')] = run.summary['r2_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2w')] = run.summary['r2_weighted_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mae')] = run.summary['mae_mean']
            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type','dataset', 'horizon', 'metric'])
mean__df = pd.DataFrame(all_data.values(), index=index)

In [500]:
mean__df.unstack(0).iloc[:, -8].isnull().sum()

0

## 1.3 不带std的结果-(baseline)

In [575]:
# 获得带+-的结果
all_data = {}

for run in runs:
        if run.state == "finished":
            try:
                # if "TNTCN" in str(run.config['model_type']):
                if "baseline" in str(run.name):
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mse')] = run.summary['mse_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2')] = run.summary['r2_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2w')] = run.summary['r2_weighted_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mae')] = run.summary['mae_mean']
            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type','dataset', 'horizon', 'metric'])
mean__df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
baseline_mean_df = mean__df.unstack(0).loc[:,0]
baseline_mean_df

model_type              Crossformer   DLinear  DeepTIMe      FiLM       GRU  \
dataset horizon metric                                                        
ETTh1   3       mae        0.314253  0.321735  0.423176  0.367544  0.333404   
                mse        0.212758  0.220075  0.358241  0.275433  0.230553   
                r2         0.688719  0.686826  0.453370  0.624022  0.665117   
                r2w        0.734981  0.725868  0.553764  0.656912  0.712815   
        6       mae        0.367815  0.369445  0.433328  0.388223  0.405782   
...                             ...       ...       ...       ...       ...   
Weather 12      r2w        0.630016  0.560484  0.320883  0.540962       NaN   
        24      mae        0.176750  0.213894  0.253416  0.199681       NaN   
                mse        0.138188  0.165067  0.243190  0.169539       NaN   
                r2         0.435156  0.383173  0.139861  0.412277       NaN   
                r2w        0.507254  0.411411  0.132844  0.395466       NaN   

model_type              Informer  Informer-t      LaST  LightTS     MTGNN  \
dataset horizon metric                                                      
ETTh1   3       mae          NaN         NaN  0.328830      NaN  0.323748   
                mse          NaN         NaN  0.222938      NaN  0.218340   
                r2           NaN         NaN  0.670281      NaN  0.668831   
                r2w          NaN         NaN  0.722301      NaN  0.728028   
        6       mae     1.875422    1.756679  0.378158      NaN  0.385105   
...                          ...         ...       ...      ...       ...   
Weather 12      r2w          NaN         NaN       NaN      NaN  0.621572   
        24      mae          NaN         NaN       NaN      NaN       NaN   
                mse          NaN         NaN       NaN      NaN       NaN   
                r2           NaN         NaN       NaN      NaN       NaN   
                r2w          NaN         NaN       NaN      NaN       NaN   

model_type               NLinear   TSMixer  TimesNet  
dataset horizon metric                                
ETTh1   3       mae     0.321303  0.343548       NaN  
                mse     0.218803  0.245335       NaN  
                r2      0.689100  0.644354       NaN  
                r2w     0.727452  0.694403       NaN  
        6       mae     0.367673  0.411966  0.404831  
...                          ...       ...       ...  
Weather 12      r2w     0.559890  0.570833       NaN  
        24      mae     0.189306  0.220565       NaN  
                mse     0.162300  0.150851       NaN  
                r2      0.445884 -1.164224       NaN  
                r2w     0.421277  0.462102       NaN  

[128 rows x 13 columns]

In [576]:
mean__df

0
model_type dataset  horizon metric          
FiLM       Weather  24      mse     0.169539
                            r2      0.412277
                            r2w     0.395466
                            mae     0.199681
NLinear    PEMS_BAY 24      mse     0.556957
...                                      ...
MTGNN      ETTh2    24      mae     0.218808
                    12      mse     0.078284
                            r2      0.292376
                            r2w     0.405592
                            mae     0.205200

[1012 rows x 1 columns]

## 1.4  不带std的结果-TNTCN

In [577]:
# 获得带+-的结果
all_data = {}
for run in runs:
        if run.state == "finished":
            try:
                if "TNTCN" in str(run.config['model_type']):
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mse')] = run.summary['mse_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2')] = run.summary['r2_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2w')] = run.summary['r2_weighted_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mae')] = run.summary['mae_mean']
            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type','dataset', 'horizon', 'metric'])
mean__df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
tntcn_mean_df = mean__df.unstack(0).loc[:,0]
tntcn_mean_df

model_type                 TNTCN  TNTCN_TCNONLY  TNTCN_fc  \
dataset horizon metric                                      
ETTh1   3       mae     0.314116       0.314904  0.315110   
                mse     0.214045       0.212932  0.214242   
                r2      0.690199       0.689274  0.685018   
                r2w     0.733378       0.734765  0.733133   
        6       mae     0.362089       0.366397  0.361402   
...                          ...            ...       ...   
Weather 12      r2w     0.615046       0.613218  0.616092   
        24      mae     0.170747       0.171549  0.167179   
                mse     0.129669       0.135849  0.128432   
                r2      0.364009       0.486079  0.388584   
                r2w     0.537631       0.515595  0.542042   

model_type              TNTCN_hetero_fastgcn_1  TNTCN_hetero_fastgcn_2  \
dataset horizon metric                                                   
ETTh1   3       mae                   0.314674                0.313758   
                mse                   0.212657                0.210209   
                r2                    0.692147                0.692860   
                r2w                   0.735108                0.738156   
        6       mae                   0.363772                0.368229   
...                                        ...                     ...   
Weather 12      r2w                        NaN                0.616020   
        24      mae                        NaN                0.180632   
                mse                        NaN                0.133742   
                r2                         NaN               -0.179104   
                r2w                        NaN                0.523110   

model_type              TNTCN_hetero_fastgcn_3  TNTCN_hetero_fastgcn_4  \
dataset horizon metric                                                   
ETTh1   3       mae                   0.313736                0.319088   
                mse                   0.211177                0.214697   
                r2                    0.692029                0.680260   
                r2w                   0.736951                0.732567   
        6       mae                   0.371872                0.363758   
...                                        ...                     ...   
Weather 12      r2w                   0.626862                0.621489   
        24      mae                   0.173527                0.180487   
                mse                   0.128777                0.132685   
                r2                    0.139697               -0.052016   
                r2w                   0.540813                0.526878   

model_type              TNTCN_hetero_fastgcn_5  \
dataset horizon metric                           
ETTh1   3       mae                   0.319382   
                mse                   0.216482   
                r2                    0.679286   
                r2w                   0.730343   
        6       mae                   0.370106   
...                                        ...   
Weather 12      r2w                   0.619923   
        24      mae                   0.178475   
                mse                   0.131995   
                r2                    0.183625   
                r2w                   0.529338   

model_type              TNTCN_hetero_fastgcn_5_adaptive  \
dataset horizon metric                                    
ETTh1   3       mae                            0.316358   
                mse                            0.213484   
                r2                             0.686637   
                r2w                            0.734078   
        6       mae                            0.368422   
...                                                 ...   
Weather 12      r2w                                 NaN   
        24      mae                                 NaN   
                mse                    

In [446]:
tntcn_mean_df.loc[:, 'TNTCN']

dataset  horizon  metric
ETTh1    3        mae       0.314116
                  mse       0.214045
                  r2        0.690199
                  r2w       0.733378
         6        mae       0.362089
                              ...   
Weather  12       r2w       0.615046
         24       mae       0.170747
                  mse       0.129669
                  r2        0.364009
                  r2w       0.537631
Name: TNTCN, Length: 128, dtype: float64

## 1.5 转化为排名


In [578]:
baseline_mean_rank_df['BiSTGNN'] = tntcn_mean_df.loc[:, 'TNTCN']

In [579]:
baseline_mean_rank_df = baseline_mean_df.copy()
baseline_mean_rank_df['BiSTGNN'] = tntcn_mean_df.loc[:, 'TNTCN_hetero_han']
baseline_model_columns =baseline_mean_rank_df.columns.get_level_values(0).difference(['dataset', 'horizon', 'metric'])
for idx, row in baseline_mean_rank_df.iterrows():
    # 只选择模型的列进行排名
    if row.name[2] in ['r2', 'r2w']:
        row[baseline_model_columns] = row[baseline_model_columns].rank(method='min', na_option='bottom', ascending=False)
    else:
        row[baseline_model_columns] = row[baseline_model_columns].rank(method='min', na_option='bottom', ascending=True)
    baseline_mean_rank_df.loc[idx, baseline_model_columns] = row[baseline_model_columns]
baseline_mean_rank_df

model_type              Crossformer  DLinear  DeepTIMe  FiLM  GRU  Informer  \
dataset horizon metric                                                        
ETTh1   3       mae             2.0      4.0      10.0   9.0  7.0      11.0   
                mse             2.0      5.0      10.0   9.0  7.0      11.0   
                r2              3.0      4.0      10.0   9.0  7.0      11.0   
                r2w             2.0      5.0      10.0   9.0  7.0      11.0   
        6       mae             3.0      4.0      11.0   7.0  9.0      13.0   
...                             ...      ...       ...   ...  ...       ...   
Weather 12      r2w             1.0      5.0       8.0   7.0  9.0       9.0   
        24      mae             1.0      5.0       7.0   4.0  8.0       8.0   
                mse             1.0      5.0       7.0   6.0  8.0       8.0   
                r2              2.0      4.0       6.0   3.0  8.0       8.0   
                r2w             1.0      5.0       7.0   6.0  8.0       8.0   

model_type              Informer-t  LaST  LightTS  MTGNN  NLinear  TSMixer  \
dataset horizon metric                                                       
ETTh1   3       mae           11.0   6.0     11.0    5.0      3.0      8.0   
                mse           11.0   6.0     11.0    3.0      4.0      8.0   
                r2            11.0   5.0     11.0    6.0      2.0      8.0   
                r2w           11.0   6.0     11.0    3.0      4.0      8.0   
        6       mae           12.0   5.0     14.0    6.0      2.0     10.0   
...                            ...   ...      ...    ...      ...      ...   
Weather 12      r2w            9.0   9.0      9.0    2.0      6.0      4.0   
        24      mae            8.0   8.0      8.0    8.0      2.0      6.0   
                mse            8.0   8.0      8.0    8.0      4.0      3.0   
                r2             8.0   8.0      8.0    8.0      1.0      7.0   
                r2w            8.0   8.0      8.0    8.0      4.0      3.0   

model_type              TimesNet  BiSTGNN  
dataset horizon metric                     
ETTh1   3       mae         11.0      1.0  
                mse         11.0      1.0  
                r2          11.0      1.0  
                r2w         11.0      1.0  
        6       mae          8.0      1.0  
...                          ...      ...  
Weather 12      r2w          9.0      3.0  
        24      mae          8.0      3.0  
                mse          8.0      2.0  
                r2           8.0      5.0  
                r2w          8.0      2.0  

[128 rows x 14 columns]

In [476]:
# baseline_mean_rank_df['BiSTGNN'].values
# baseline_mean_rank_df

model_type              Crossformer  DLinear  DeepTIMe  FiLM  GRU  Informer  \
dataset horizon metric                                                        
ETTh1   3       mae             2.0      4.0      10.0   9.0  7.0      11.0   
                mse             2.0      5.0      10.0   9.0  7.0      11.0   
                r2              3.0      4.0      10.0   9.0  7.0      11.0   
                r2w             2.0      5.0      10.0   9.0  7.0      11.0   
        6       mae             3.0      4.0      11.0   7.0  9.0      13.0   
...                             ...      ...       ...   ...  ...       ...   
Weather 12      r2w             1.0      5.0       7.0   8.0  8.0       8.0   
        24      mae             1.0      4.0       6.0   7.0  7.0       7.0   
                mse             1.0      5.0       6.0   7.0  7.0       7.0   
                r2              2.0      3.0       5.0   7.0  7.0       7.0   
                r2w             1.0      5.0       6.0   7.0  7.0       7.0   

model_type              Informer-t  LaST  LightTS  MTGNN  NLinear  TSMixer  \
dataset horizon metric                                                       
ETTh1   3       mae           11.0   6.0     11.0    5.0      3.0      8.0   
                mse           11.0   6.0     11.0    3.0      4.0      8.0   
                r2            11.0   5.0     11.0    6.0      2.0      8.0   
                r2w           11.0   6.0     11.0    3.0      4.0      8.0   
        6       mae           12.0   5.0     14.0    6.0      2.0     10.0   
...                            ...   ...      ...    ...      ...      ...   
Weather 12      r2w            8.0   8.0      8.0    2.0      6.0      4.0   
        24      mae            7.0   7.0      7.0    7.0      2.0      5.0   
                mse            7.0   7.0      7.0    7.0      4.0      3.0   
                r2             7.0   7.0      7.0    7.0      1.0      6.0   
                r2w            7.0   7.0      7.0    7.0      4.0      3.0   

model_type              TimesNet  BiSTGNN  
dataset horizon metric                     
ETTh1   3       mae         11.0      1.0  
                mse         11.0      1.0  
                r2          11.0      1.0  
                r2w         11.0      1.0  
        6       mae          8.0      1.0  
...                          ...      ...  
Weather 12      r2w          8.0      3.0  
        24      mae          7.0      3.0  
                mse          7.0      2.0  
                r2           7.0      4.0  
                r2w          7.0      2.0  

[128 rows x 14 columns]

In [477]:
baseline_mean_rank_df.to_excel('rank.xlsx')

In [580]:
tntcn_mean_rank_df = tntcn_mean_df.copy()
model_columns =tntcn_mean_df.columns.get_level_values(0).difference(['dataset', 'horizon', 'metric'])
    
for idx, row in tntcn_mean_rank_df.iterrows():
    # 只选择模型的列进行排名
    row[model_columns] = row[model_columns].rank(method='min', na_option='bottom', ascending=True)
    mean_rank_df.loc[idx, model_columns] = row[model_columns]
tntcn_mean_rank_df

model_type              TNTCN  TNTCN_TCNONLY  TNTCN_fc  \
dataset horizon metric                                   
ETTh1   3       mae       3.0            7.0       9.0   
                mse      10.0            5.0      12.0   
                r2       14.0           12.0       6.0   
                r2w       9.0           14.0       7.0   
        6       mae       5.0           10.0       2.0   
...                       ...            ...       ...   
Weather 12      r2w       6.0            4.0      10.0   
        24      mae       4.0            5.0       1.0   
                mse       6.0           11.0       1.0   
                r2       11.0           15.0      13.0   
                r2w      10.0            5.0      15.0   

model_type              TNTCN_hetero_fastgcn_1  TNTCN_hetero_fastgcn_2  \
dataset horizon metric                                                   
ETTh1   3       mae                        5.0                     2.0   
                mse                        4.0                     2.0   
                r2                        16.0                    17.0   
                r2w                       15.0                    17.0   
        6       mae                        8.0                    11.0   
...                                        ...                     ...   
Weather 12      r2w                       17.0                     9.0   
        24      mae                       16.0                    11.0   
                mse                       16.0                    10.0   
                r2                        16.0                     3.0   
                r2w                       16.0                     6.0   

model_type              TNTCN_hetero_fastgcn_3  TNTCN_hetero_fastgcn_4  \
dataset horizon metric                                                   
ETTh1   3       mae                        1.0                    15.0   
                mse                        3.0                    14.0   
                r2                        15.0                     4.0   
                r2w                       16.0                     5.0   
        6       mae                       14.0                     7.0   
...                                        ...                     ...   
Weather 12      r2w                       16.0                    14.0   
        24      mae                        6.0                    10.0   
                mse                        4.0                     9.0   
                r2                         7.0                     5.0   
                r2w                       12.0                     7.0   

model_type              TNTCN_hetero_fastgcn_5  \
dataset horizon metric                           
ETTh1   3       mae                       16.0   
                mse                       16.0   
                r2                         3.0   
                r2w                        3.0   
        6       mae                       13.0   
...                                        ...   
Weather 12      r2w                       13.0   
        24      mae                        9.0   
                mse                        8.0   
                r2                         8.0   
                r2w                        8.0   

model_type              TNTCN_hetero_fastgcn_5_adaptive  \
dataset horizon metric                                    
ETTh1   3       mae                                14.0   
                mse                                 8.0   
                r2                                 11.0   
                r2w                                11.0   
        6       mae                                12.0   
...                                                 ...   
Weather 12      r2w                                17.0   
        24      mae                                16.0   
                mse                                16.0   
                r2 

In [380]:

for metric in df.index.get_level_values('metric').unique():
    metric_data = df.xs(metric, level='metric').loc[:, model_columns]
    # 计算排名，这里假设数值越小排名越前
    ranks = metric_data.rank(axis=1, method='min', na_option='bottom', ascending=True)
    # 输出每个metric的排名
    print(f"\nRanking for {metric}:")
    print(ranks)

    
    
    
# 对DataFrame的每一行计算排名，并替换原值
for idx, row in df.iterrows():
    # 只选择模型的列进行排名
    row[model_columns] = row[model_columns].rank(method='min', na_option='bottom', ascending=True)
    df.loc[idx, model_columns] = row[model_columns]



Ranking for mae:
model_type            Crossformer  DLinear  DeepTIMe  FiLM   GRU  Informer  \
dataset      horizon                                                         
ETTh1        3                1.0      3.0       9.0   8.0   6.0      10.0   
             6                2.0      3.0      10.0   6.0   8.0      12.0   
             12               3.0      2.0       9.0   6.0   7.0      12.0   
             24               4.0      1.0       9.0   6.0   7.0      12.0   
ETTh2        3                4.0      2.0      10.0   9.0   5.0      11.0   
             6                5.0      1.0       9.0   7.0   4.0      10.0   
             12               5.0      2.0       9.0   6.0   3.0      11.0   
             24               6.0      2.0       7.0   3.0   9.0      10.0   
ETTm1        3                4.0      6.0      10.0   9.0  11.0      12.0   
             6                2.0      4.0       8.0   7.0   9.0      10.0   
             12               1.0      4.0    

# 2 数据处理与导出

## 2.1 获得 diff 之后的数据

In [139]:
# 获得带+-的结果
all_data = {}
for run in runs:
        if run.state == "finished":
            try:
                if "TNTCN_hetero_han" == str(run.config['model_type']) or  "BiSTGNN" in str(run.config['model_type']) and run.config['pred_len'] < 3:
                # if "TNTCN" in str(run.config['model_type']):
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mse')] = run.summary['mse_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2')] = run.summary['r2_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'r2w')] = run.summary['r2_weighted_mean']
                    all_data[(run.config['model_type'],run.config['dataset_type'], run.config['horizon'], 'mae')] = run.summary['mae_mean']
            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type', 'dataset', 'horizon', 'metric'])
all_df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
result_df = all_df.unstack(0).loc[:,0]
df_pivot = result_df.pivot_table(index=['dataset', 'horizon'], columns='metric')
df_pivot

model_type           BiSTGNNv4                               BiSTGNNv5  \
metric                     mae       mse        r2       r2w       mae   
dataset      horizon                                                     
ETTh1        3        0.306588  0.206078  0.698532  0.743302       NaN   
             6        0.355840  0.270391  0.612260  0.662732       NaN   
             12       0.377297  0.293748  0.560503  0.631547       NaN   
             24       0.394263  0.314391  0.503821  0.604342       NaN   
ETTh2        3        0.156067  0.046208  0.550902  0.647989       NaN   
             6        0.187543  0.066555  0.406587  0.493139       NaN   
             12       0.202804  0.076749  0.324409  0.417251       NaN   
             24       0.228795  0.093289  0.134360  0.294913       NaN   
ETTm1        3        0.209973  0.112417  0.846130  0.866735       NaN   
             6        0.255010  0.152444  0.790098  0.819280       NaN   
             12       0.305277  0.206743  0.714302  0.754993       NaN   
             24       0.357434  0.273511  0.622728  0.674795       NaN   
ETTm2        3        0.115858  0.027001  0.717118  0.795098       NaN   
             6        0.135316  0.035784  0.631785  0.728423       NaN   
             12       0.153307  0.045786  0.549726  0.652430       NaN   
             24       0.181856  0.064517  0.421843  0.509506       NaN   
ExchangeRate 3        0.051338  0.005101  0.926604  0.955612       NaN   
             6        0.069503  0.008887  0.866400  0.922831       NaN   
             12       0.101037  0.018439  0.735217  0.840576       NaN   
             24       0.136609  0.032262  0.522999  0.721956  0.138388   
METR_LA      1             NaN       NaN       NaN       NaN       NaN   
             3             NaN       NaN       NaN       NaN       NaN   
             6             NaN       NaN       NaN       NaN       NaN   
             12            NaN       NaN       NaN       NaN       NaN   
             24            NaN       NaN       NaN       NaN       NaN   
PEMS_BAY     1             NaN       NaN       NaN       NaN       NaN   
             3             NaN       NaN       NaN       NaN       NaN   
             6             NaN       NaN       NaN       NaN       NaN   
             12            NaN       NaN       NaN       NaN       NaN   
             24            NaN       NaN       NaN       NaN       NaN   
PeMS_D7      6        0.266050  0.226100  0.699003  0.780226       NaN   
             12       0.311843  0.305358  0.293448  0.703694       NaN   
             24       0.339392  0.346770  0.247220  0.662061       NaN   
SolarEnergy  3        0.079478  0.027330  0.958472  0.958590       NaN   
             12       0.157814  0.092227  0.859088  0.860426       NaN   
Weather      3        0.086228  0.067534  0.739777  0.758714       NaN   
             6        0.101040  0.083718  0.727087  0.700990       NaN   
             12       0.130284  0.105619  0.566397  0.622981       NaN   
             24       0.205511  0.150097  0.112683  0.464792       NaN   

model_type                                         TNTCN_hetero_han            \
metric                     mse        r2       r2w              mae       mse   
dataset      horizon                                                            
ETTh1        3             NaN       NaN       NaN         0.314124  0.209184   
             6             NaN       NaN       NaN         0.359600  0.267637   
             12            NaN       NaN       NaN         0.377893  0.292929   
             24            NaN       NaN       NaN         0.386093  0.310093   
ETTh2        3             NaN       NaN       NaN         0.157896  0.047380   
             6             NaN       NaN       NaN         0.182259  0.064140   
             12            NaN       NaN       NaN         0.200595  0.075843   
             24            NaN       NaN       NaN         0.213369  0.085140   


In [140]:
# (df_pivot.iloc[:,2] - df_pivot.iloc[:,1] )
reverse_metrics = ['r2', 'r2w']

model_types =  result_df.columns # ['TNTCN_hetero_han_homo'] #

base_model = 'TNTCN_hetero_han'

model_exclude_baseline = set(model_types) - set([base_model])
print(model_exclude_baseline)
total_metrics = 4
df_pivot[base_model]

for model in model_exclude_baseline:
    for metric in ['r2', 'r2w', 'mae', 'mse']:
        baseline_value = df_pivot[base_model][metric]
        model_value = df_pivot[model][metric]
        if metric not in ['r2', 'r2w']:
            df_pivot.loc[:, (model, metric)] =  (baseline_value - model_value)/baseline_value 
        else:
            df_pivot.loc[:, (model, metric)] =  (model_value - baseline_value)/baseline_value 
mean_diff_percent_df = df_pivot.stack().iloc[:,0:-1]

{'BiSTGNNv4', 'BiSTGNNv5'}


In [142]:
mean_diff_percent_df

model_type              BiSTGNNv4  BiSTGNNv5
dataset horizon metric                      
ETTh1   3       mae      0.023990        NaN
                mse      0.014845        NaN
                r2       0.004587        NaN
                r2w      0.005231        NaN
        6       mae      0.010454        NaN
...                           ...        ...
Weather 12      r2w      0.011058        NaN
        24      mae     -0.058567        NaN
                mse     -0.045829        NaN
                r2      -0.659136        NaN
                r2w     -0.048036        NaN

[134 rows x 2 columns]

In [135]:
result_df

model_type              BiSTGNNv4  BiSTGNNv5  TNTCN_hetero_han
dataset horizon metric                                        
ETTh1   3       mae      0.306588        NaN          0.314124
                mse      0.206078        NaN          0.209184
                r2       0.698532        NaN          0.695343
                r2w      0.743302        NaN          0.739434
        6       mae      0.355840        NaN          0.359600
...                           ...        ...               ...
Weather 12      r2w      0.622981        NaN          0.616168
        24      mae      0.205511        NaN          0.194141
                mse      0.150097        NaN          0.143519
                r2       0.112683        NaN          0.330580
                r2w      0.464792        NaN          0.488245

[150 rows x 3 columns]

## 2 conditional formating

In [143]:
# 添加 condition formating
import xlsxwriter


writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
# Convert the dataframe to an XlsxWriter Excel object.
mean_diff_percent_df.to_excel(writer, sheet_name='Sheet1')

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']



rows = len(mean_diff_percent_df)


start_col = 3
model_types = list(mean_diff_percent_df.columns)
end_col = start_col + len(model_types) - 1

start_col_letter =  xlsxwriter.utility.xl_col_to_name(start_col)
end_col_letter =  xlsxwriter.utility.xl_col_to_name(end_col)



# 调整百分比格式
percent_format = workbook.add_format({'num_format': '0.00%'})
for col_num in range(start_col, end_col+1):
    # 调整列宽以适应内容
    column_width = len(model_types[col_num - start_col])
    worksheet.set_column(col_num, col_num, column_width, percent_format)


for i in range(rows):
    col_letter = xlsxwriter.utility.xl_col_to_name(i)
    worksheet.conditional_format(f'{start_col_letter}{i}:{end_col_letter}{i}', 
                                 {'type': 'data_bar',
                                  'bar_negative_color': 'red',
                                  'bar_color': '#32CD32',#'green',
                                  'bar_direction': 'left',
                                  'bar_axis_position': 'middle',
                                  'bar_solid': True,
                                   'min_type':'min',
                                   'max_type':'max',

#                                  # 'data_bar_2010': True,
                                 }
                                )
    


## 2.3 导出数据

In [144]:
writer.close()

# 3 导出 论文需要的数据


## 3.1 单步预测

最好结果用黑体

In [17]:
'_'.join(['asdf', 'asdf', 0])

TypeError: sequence item 2: expected str instance, int found

In [20]:
# models = ["GRU","DLinear", "NLinear", "TSMixer","DeepTIMe","LaST", "Crossformer", "FiLM", "TNTCN_hetero_han","MTGNN", "TNTCN_bistgnn_han"] #  "MICN",
models = ["TSMixer","DeepTIMe","LaST", "Crossformer", "NLinear", "DLinear", "FiLM","MTGNN", "HSTGNNv3"] #  "MICN",
datasets = ["ETTm1", "ETTm2", "ETTh1", "ETTh2","ExchangeRate", "Weather","SolarEnergy","ILI","SP500","Traffic"] #"ExchangeRate",

# 获得带+-的结果
all_data = {}
for run in runs:
        if run.state == "finished" or run.state == "running":
            try:
                if str(run.config['model_type']) in models and run.config['dataset_type'] in datasets and run.config['pred_len'] == 1:
                    model_name = run.config['model_type']
                    if model_name == 'HSTGNNv3':
                        model_name = '_'.join([model_name, run.config['graph_build_type'], str(run.config['d0'])])  
                    
                    all_data[(model_name,run.config['dataset_type'], run.config['horizon'], 'mse')] = run.summary['mse_mean']
                    all_data[(model_name,run.config['dataset_type'], run.config['horizon'], 'r2')] = run.summary['r2_mean']
                    all_data[(model_name,run.config['dataset_type'], run.config['horizon'], 'r2w')] = run.summary['r2_weighted_mean']
                    all_data[(model_name,run.config['dataset_type'], run.config['horizon'], 'mae')] = run.summary['mae_mean']
                
            
            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type', 'dataset', 'horizon', 'metric'])
all_df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
result_df = all_df.unstack(0).loc[:,0]
result_df
# # result_df.to_excel('results.xlsx')
df_pivot = result_df.pivot_table(index=['dataset', 'horizon'], columns='metric')
# df_pivot
output = df_pivot.loc[:, (slice(None), ['mae', 'mse'])].round(4)
output = output.applymap(lambda x: f'{x:.4f}')

output.to_excel('./results.xlsx')
output

model_type           Crossformer         DLinear         DeepTIMe          \
metric                       mae     mse     mae     mse      mae     mse   
dataset      horizon                                                        
ETTh1        3            0.3143  0.2128  0.3217  0.2201   0.4232  0.3582   
             6            0.3678  0.2832  0.3694  0.2811   0.4333  0.3683   
             12           0.3848  0.3065  0.3834  0.3037   0.4404  0.3809   
             24           0.4071  0.3357  0.3820  0.3129   0.4647  0.4154   
ETTh2        3            0.1611  0.0488  0.1574  0.0475   0.2092  0.0820   
             6            0.1896  0.0672  0.1831  0.0647   0.2186  0.0894   
             12           0.2072  0.0802  0.1963  0.0747   0.2285  0.0983   
             24           0.2376  0.1014  0.2060  0.0815   0.2431  0.1109   
ETTm1        3            0.2144  0.1168  0.2163  0.1163   0.3587  0.2699   
             6            0.2635  0.1581  0.2665  0.1600   0.3883  0.3116   
             12           0.3182  0.2215  0.3252  0.2263   0.4180  0.3577   
             24           0.3661  0.2861  0.3735  0.2918   0.4402  0.3950   
ETTm2        3            0.1167  0.0277  0.1160  0.0274   0.1730  0.0563   
             6            0.1374  0.0369  0.1359  0.0367   0.1898  0.0676   
             12           0.1561  0.0473  0.1592  0.0497   0.2031  0.0779   
             24           0.1876  0.0673  0.1871  0.0685   0.2162  0.0889   
ExchangeRate 3            0.0886  0.0177  0.0443  0.0039   0.0593  0.0066   
             6            0.0932  0.0182  0.0642  0.0077   0.0776  0.0110   
             12           0.1263  0.0314  0.0885  0.0144   0.1072  0.0206   
             24           0.1931  0.0678  0.1205  0.0259   0.1450  0.0380   
ILI          3               nan     nan  0.5043  0.3717      nan     nan   
             6               nan     nan  0.8974  1.1511      nan     nan   
             12              nan     nan  1.3217  2.4133      nan     nan   
             24              nan     nan  1.7240  3.6741      nan     nan   
SP500        3               nan     nan  0.4710  0.5820      nan     nan   
             6               nan     nan  0.4816  0.6001      nan     nan   
             12              nan     nan  0.5005  0.6332      nan     nan   
             24              nan     nan  0.5305  0.6886      nan     nan   
SolarEnergy  3               nan     nan  0.1112  0.0359   0.3698  0.3272   
             6               nan     nan  0.1741  0.0751   0.4458  0.4516   
             12              nan     nan  0.2509  0.1389   0.5446  0.6626   
             24              nan     nan  0.3216  0.2197   0.6163  0.8090   
Traffic      3               nan     nan     nan     nan   0.2902  0.3240   
             6               nan     nan     nan     nan   0.2930  0.3273   
             12              nan     nan     nan     nan   0.2918  0.3250   
             24              nan     nan     nan     nan   0.2938  0.3287   
Weather      3            0.0822  0.0649  0.0874  0.0694   0.1548  0.1260   
             6            0.1000  0.0821  0.1163  0.0915   0.1782  0.1522   
             12           0.1295  0.1036  0.1533  0.1231   0.2128  0.1903   
             24           0.1768  0.1382  0.2139  0.1651   0.2534  0.2432   

model_type              FiLM         HSTGNNv3_adaptive_2          ...  \
metric                   mae     mse                 mae     mse  ...   
dataset      horizon                                              ...   
ETTh1        3        0.3675  0.2754                 nan     nan  ...   
             6        0.3882  0.3046                 nan     nan  ...   
             12       0.3960  0.3189                 nan     nan  ...   
             24       0.4264  0.3606                 nan     nan  ...   
ETTh2        3        0.1879  0.0674                 nan     nan  ...   
             6        0.1980  0.0757                 nan     nan  ...   
             12       0.2074  0

In [109]:
# # 假设 df 是你的 DataFrame
# # 创建一个 ExcelWriter 对象
# with pd.ExcelWriter('./output.xlsx') as writer:
#     # 定义一个函数，用于找到最好和第二好的结果，并将它们设置为黑体
#     def highlight_best(s):
#         # 分离 mae 和 mse
#         # print(s)
#         # 分离 mae 和 mse
#         mae = s.xs('mae', level='metric')
#         mse = s.xs('mse', level='metric')
        
#         # 找到 mae 和 mse 的最好和第二好的结果
#         is_best_mae = mae == mae.min()
#         is_second_best_mae = mae.rank(method='min') == 2
#         is_best_mse = mse == mse.min()
#         is_second_best_mse = mse.rank(method='min') == 2
#         # is_best = is_best_mse | is_best_mae
#         if s.name[0] == 'Weather' and s.name[1] ==3:
#             print(is_best_mae)
#         if s.name[0] == 'Weather' and s.name[1] ==3:
#             print(is_best_mse)

#         # 交错合并
#         is_best_combined = [val for pair in zip(is_best_mae, is_best_mse) for val in pair]
#         is_second_best_combined = [val for pair in zip(is_second_best_mae, is_second_best_mse) for val in pair]
        
#         return ['font-weight: bold' if v else '' for v in is_best_combined ]
#         # return ['color: black; font-weight: bold' if v else 'color: darkgreen;  font-weight: bold' if sb else '' for v, sb in zip(is_best, is_second_best)]
#         # return ['color: black; font-weight: bold' if v else 'color: darkgreen;  font-weight: bold' if sb else '' for v, sb in zip(is_best, is_second_best)]

#     # 对 DataFrame 的每一列应用 highlight_best 函数
#     styled_df = output.style.apply(highlight_best, axis=1)

#     # 将样式化的 DataFrame 写入 Excel 文件
#     styled_df.to_excel(writer, engine='openpyxl')


## 3.2 多步预测


In [3]:
# 获得带+-的结果
models = ["GMAN","DCRNN","STGCN", "GraphWavenet", "MTGNN", "BiSTGNNv6"] #  "MICN",
multistep_datasets = [ "PeMS_D7",  "PEMS_BAY", "METR_LA", "PEMS03", "PEMS04", "PEMS07", "PEMS08"]
horizons = [3,6,12]
all_data = {}
for run in runs:
        if run.state == "finished" or run.state == "running":
            try:
                if run.config['dataset_type'] in multistep_datasets and  run.config['pred_len'] >2 and run.config['model_type'] in models and run.config['pred_len'] in horizons:
                # print(run.config['model_type'],run.config['dataset_type'],run.config['pred_len'])
                    name = run.config['model_type']
                    if run.config['model_type'] == "BiSTGNNv6":
                        if run.config['gcn_type'] == 'weighted_han' and run.config['graph_build_type'] == 'predefined_adaptive': 
                            name = 'BiSTGNN'
                        else:
                            continue
                        # if run.config['gcn_type'] == 'weighted_han_homo':
                        #     name = 'homo_only'
                        # elif run.config['gcn_type'] == 'weighted_han_hetero':
                        #     name = 'hetero_only'
                        # elif run.config['graph_build_type'] == 'fully_connected':
                        #     name = 'fully_connected'
                            

                    all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'rmse')] = run.summary['rmse_mean']
                    all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mape')] = run.summary['mape_mean']
                    all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mae')] = run.summary['mae_mean']

            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type','dataset', 'horizon', 'metric'])
all_df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
result_df = all_df.unstack(0).loc[:,0]
result_df.to_excel('results.xlsx')
result_df

model_type               BiSTGNN      DCRNN      GMAN  GraphWavenet     MTGNN  \
dataset horizon metric                                                          
METR_LA 3       mae     3.548780   8.629914  4.089411      3.454876  3.289390   
                mape    0.077590   0.101861  0.085727      0.078213  0.073027   
                rmse    7.824927  15.324094  8.283894      7.870789  7.653911   
        6       mae     4.597614   9.081003  5.121222      4.627012  4.629796   
                mape    0.096077   0.113104  0.105383      0.097172  0.093833   
...                          ...        ...       ...           ...       ...   
PeMS_D7 6       mape    0.056393   0.135282  0.069076      0.058947  0.055540   
                rmse    4.295256   7.476990  5.078294      4.639217  4.211556   
        12      mae     2.956619   4.911012  3.554961      3.406156  2.891180   
                mape    0.071834   0.146042  0.087801      0.084629  0.070375   
                rmse    5.362164   8.157394  6.230018      6.217917  5.242207   

model_type                 STGCN  
dataset horizon metric            
METR_LA 3       mae     4.384566  
                mape    0.091033  
                rmse    9.025284  
        6       mae     5.438525  
                mape    0.105450  
...                          ...  
PeMS_D7 6       mape    0.111005  
                rmse    7.185563  
        12      mae     4.444277  
                mape    0.111888  
                rmse    7.312718  

[63 rows x 6 columns]

In [4]:
df_pivot = result_df.pivot_table(index=['dataset', 'horizon'], columns='metric')
df_pivot

model_type          BiSTGNN                           DCRNN            \
metric                  mae      mape       rmse        mae      mape   
dataset  horizon                                                        
METR_LA  3         3.548780  0.077590   7.824927   8.629914  0.101861   
         6         4.597614  0.096077   9.530847   9.081003  0.113104   
         12        5.976110  0.118134  11.676210        NaN       NaN   
PEMS03   3        14.103537  0.169999  22.780492  14.846891  0.167453   
         6        15.399016  0.179822  25.140778  16.808025  0.205450   
         12       17.110492  0.201268  28.516004  20.583715  0.281975   
PEMS04   3        18.260696  0.123051  29.143975  40.201955  0.558785   
         6        19.279240  0.132860  30.537230  41.219057  0.561162   
         12       20.445175  0.139944  32.429371  43.491263  0.569641   
PEMS07   3        19.238273  0.082592  30.296657  41.797844  0.340067   
         6        20.872215  0.088951  32.718488  43.495829  0.346935   
         12       23.070272  0.101206  36.125587  47.019269  0.363165   
PEMS08   3        14.130773  0.093537  21.716567  33.483614  0.342009   
         6        15.212300  0.101731  23.335616  34.413346  0.346709   
         12       16.615902  0.111404  25.641779  36.269691  0.356824   
PEMS_BAY 3         1.245401  0.026011   2.556193   2.592901  0.085068   
         6         1.561488  0.034311   3.286866   2.776464  0.088395   
         12        1.920183  0.044545   4.076958   3.054685  0.093705   
PeMS_D7  3              NaN       NaN        NaN   4.130346  0.129266   
         6         2.381270  0.056393   4.295256   4.415707  0.135282   
         12        2.956619  0.071834   5.362164   4.911012  0.146042   

model_type                        GMAN                      GraphWavenet  \
metric                 rmse        mae      mape       rmse          mae   
dataset  horizon                                                           
METR_LA  3        15.324094   4.089411  0.085727   8.283894     3.454876   
         6        15.798770   5.121222  0.105383   9.860480     4.627012   
         12             NaN   6.498656  0.128076  11.780351     5.946664   
PEMS03   3        23.861248  17.253944  0.268893  27.180214    13.763937   
         6        27.326003  18.278421  0.234419  28.868594    14.627737   
         12       33.578512  21.059885  0.295382  32.674142    16.871732   
PEMS04   3        56.141273  21.888972  0.164244  33.512587    18.702035   
         6        57.184486  23.441880  0.172659  35.694960    19.975512   
         12       59.753181  25.990612  0.194587  39.266713    21.966678   
PEMS07   3        60.119137  21.973640  0.099630  33.467593          NaN   
         6        62.004280  25.438301  0.122329  37.778529    20.765542   
         12       66.040877  28.936219  0.143463  42.481323    23.630854   
PEMS08   3        47.133704  16.263210  0.110359  24.731210    14.019139   
         6        48.174228  18.071028  0.123177  27.397014    14.723381   
         12       50.322453  20.117547  0.141140  30.367360    16.131944   
PEMS_BAY 3         6.389820   1.450157  0.031999   2.933205     1.266671   
         6         6.545117   1.774358  0.040194   3.656920     1.596202   
         12        6.829294   2.277151  0.054395   4.724143     2.051612   
PeMS_D7  3         7.091934   2.065779  0.047679   3.606767     1.872953   
         6         7.476990   2.874662  0.069076   5.078294     2.486459   
         12        8.157394   3.554961  0.087801   6.230018     3.406156   

model_type                                 MTGNN                       \
metric                mape       rmse        mae      mape       rmse   
dataset  horizon                                                        
METR_LA  3        0.078213   7.870789   3.289390  0.073027   7.653911   
         6        0.097172   9.396866   4.629796  0.093833   9.292840   
         12       0.123091  11.404235   5.630406  0.11056

In [157]:
models

['GMAN', 'DCRNN', 'STGCN', 'GraphWavenet', 'MTGNN', 'BiSTGNNv6']

## 3.3 消融实验数据

### 1. 单步预测

In [236]:
## 1.2 读取 mean 数据


from dataclasses import asdict
import string
from openpyxl import Workbook
import pandas as pd
import wandb
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment

from torch_timeseries.experiments.dlinear_experiment import DLinearExperiment
# 初始化Wandb API
api = wandb.Api()

def convert_dict(dictionary):
    converted_dict = {}
    for key, value in dictionary.items():
        converted_dict[f"config.{key}"] = value
    return converted_dict

# 查询特定配置的运行
project_name = "BiSTGNN"  # 替换为你的项目名称
config_name = "your-config-name"  # 替换为你的配置名称

# single steps report

models = ["BiSTGNNv6"]
datasets = ["ExchangeRate",  "ETTm1", "ETTm2", "ETTh1", "ETTh2", "Weather", "SolarEnerge"] #"ExchangeRate",
multistep_datasets = [ "PeMS_D7",  "PEMS_BAY", "METR_LA"]
horizons = [3, 6, 12, 24]

all_data = {}
runs = api.runs(path="BiSTGNN")


In [240]:
# 获得带+-的结果
all_data = {}
datasets = ["ExchangeRate",  "ETTm1", "ETTm2", "ETTh1", "ETTh2", "Weather", "SolarEnerge"] #"ExchangeRate",

for run in runs:
        if run.state == "finished" or run.state == "running":
            try:
                if (run.config['dataset_type'] in datasets and  run.config['pred_len'] <2 )  :
                    # if "TNTCN" in str(run.config['model_type']):
                    if "BiSTGNNv6" in str(run.config['model_type']) or 'TNTCN_TCNONLY' in str(run.config['model_type']) :
                        if run.config['gcn_type'] == 'weighted_han_homo':
                            name = 'homo_only'
                        elif run.config['gcn_type'] == 'weighted_han_hetero':
                            name = 'hetero_only'
                        elif run.config['graph_build_type'] == 'fully_connected':
                            name = 'fully_connected'
                        else:
                            name = run.config['model_type']

                        all_data[(name,run.config['dataset_type'], run.config['horizon'], 'mse')] = run.summary['mse_mean']
                        all_data[(name,run.config['dataset_type'], run.config['horizon'], 'r2')] = run.summary['r2_mean']
                        all_data[(name,run.config['dataset_type'], run.config['horizon'], 'r2w')] = run.summary['r2_weighted_mean']
                        all_data[(name,run.config['dataset_type'], run.config['horizon'], 'mae')] = run.summary['mae_mean']

            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type','dataset', 'horizon', 'metric'])
all_df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
result_df = all_df.unstack(0).loc[:,0]
result_df.to_excel('results.xlsx')

### 2. 多步预测

In [27]:
# 获得带+-的结果
models = ["TSMixer","DeepTIMe","LaST", "Crossformer", "FiLM","MTGNN", "BiSTGNNv6"] #  "MICN",
multistep_datasets = [ "PeMS_D7",  "PEMS_BAY", "METR_LA", "PEMS03","PEMS04","PEMS07","PEMS08"]

all_data = {}
for run in runs:
        if run.state == "finished" or run.state == "running":
            try:
                if (run.config['dataset_type'] in multistep_datasets and  run.config['pred_len'] >2):
                        if "BiSTGNNv6" in str(run.config['model_type']) or 'TNTCN_TCNONLY' in str(run.config['model_type']) :
                            if run.config['gcn_type'] == 'weighted_han_homo':
                                name = 'homo_only'
                            elif run.config['gcn_type'] == 'weighted_han_hetero':
                                name = 'hetero_only'
                            elif run.config['graph_build_type'] == 'fully_connected':
                                name = 'fully_connected'
                            elif run.config['without_gc'] == 'without_gc':
                                name = 'without_gc'

                            else:
                                name = run.config['model_type']
                            all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mse')] = run.summary['mse_mean']
                            all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'rmse')] = run.summary['rmse_mean']
                            all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mape')] = run.summary['mape_mean']
                            all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mae')] = run.summary['mae_mean']

            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['model_type','dataset', 'horizon', 'metric'])
all_df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
result_df = all_df.unstack(0).loc[:,0]
result_df.to_excel('results.xlsx')
result_df

model_type              fully_connected  hetero_only  homo_only
dataset horizon metric                                         
METR_LA 3       mae            3.579419     3.509607   3.488463
                mape           0.078440     0.077224   0.077288
                mse            0.169902    61.311044  61.175907
                rmse           7.897504     7.830102   7.821467
        6       mae                 NaN     4.235427   4.756249
...                                 ...          ...        ...
PeMS_D7 6       rmse           4.303672     4.308558   4.286740
        12      mae            3.003128     2.990238   2.984709
                mape           0.072616     0.073839   0.072359
                mse            0.220662    29.728799  29.350558
                rmse           5.386747     5.452259   5.417358

[84 rows x 3 columns]

## 3.4 消融实验数据

In [38]:
# 获得带+-的结果
def config_matches(run_config, config_dict):
    for key, value in config_dict.items():
        if key not in run_config or run_config[key] != value:
            return False
    return True

config_dict =  {
    "model_type": "BiSTGNNv6",
    "output_layer_type": "tcn6",
    "latent_dim": 32,
    "gcn_layers":2,
    "heads": 1,
    
    "tcn_channel": 16,
    "self_loop_eps":0.1,
    "self_loop_eps":0.5,
    "tcn_layers":3,
}


all_data = {}
for run in runs:
        if run.state == "finished":
            try:
                # if "TNTCN" in str(run.config['model_type']):
                if  config_matches(run.config, config_dict):
                    if run.config['gcn_type'] == 'weighted_han_homo':
                        name = 'homo_only'
                    elif run.config['gcn_type'] == 'weighted_han_hetero':
                        name = 'hetero_only'
                    elif run.config['graph_build_type'] == 'fully_connected':
                        name = 'fully_connected'
                    else:
                        name = run.config['model_type']
                    all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mse')] = run.summary['mse_mean']
                    all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'rmse')] = run.summary['rmse_mean']
                    all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mape')] = run.summary['mape_mean']
                    all_data[(name,run.config['dataset_type'], run.config['pred_len'], 'mae')] = run.summary['mae_mean']
            except:
                continue
index = pd.MultiIndex.from_tuples(all_data.keys(), names=['dataset', 'horizon', 'metric', 'model_type'])
all_df = pd.DataFrame(all_data.values(), index=index)
"""
(Pdb) df
                                               0
dataset horizon metric model_type               
GRU     ETTh1   12     mse         0.3308±0.0058
                       r2          0.5113±0.0118
                       r2w         0.5851±0.0072
                       mae         0.4093±0.0041
DLinear ETTh2   12     mse         0.0747±0.0007
...                                          ...
MTGNN   ETTh2   24     mae         0.2188±0.0054
                12     mse         0.0783±0.0036
                       r2          0.2924±0.0532
                       r2w         0.4056±0.0272
                       mae         0.2052±0.0065

"""
# .loc[:,0] 去掉第一列索引 （0）
result_df = all_df.unstack(0).loc[:,0]
result_df.to_excel('results.xlsx')

In [39]:
result_df

dataset                         BiSTGNNv6  fully_connected  hetero_only  \
horizon      metric model_type                                            
ETTh1        1      mse          0.210427         0.211744     0.298470   
ETTh2        1      mse          0.046651         0.075536     0.046742   
ETTm1        1      mse          0.154678         0.154198     0.113974   
ETTm2        1      mse          0.026867         0.065483     0.036146   
ExchangeRate 1      mse          0.007888         0.007889     0.027529   
METR_LA      3      mse          0.168742         0.169902          NaN   
             6      mse          0.239999         0.241359          NaN   
             12     mse          0.364865              NaN          NaN   
PEMS_BAY     3      mse          0.118763              NaN          NaN   
             6      mse          0.178997         0.179850          NaN   
             12     mse          0.260608         0.260298          NaN   
PeMS_D7      3      mse          0.095415         0.094468          NaN   
             6      mse          0.149284         0.147279          NaN   
             12     mse          0.219403         0.220662          NaN   
SolarEnergy  1      mse          0.050024              NaN     0.154399   
Weather      1      mse          0.066229         0.144295     0.066177   

dataset                         homo_only  
horizon      metric model_type             
ETTh1        1      mse          0.296402  
ETTh2        1      mse          0.087580  
ETTm1        1      mse          0.209251  
ETTm2        1      mse          0.026916  
ExchangeRate 1      mse          0.015163  
METR_LA      3      mse               NaN  
             6      mse               NaN  
             12     mse               NaN  
PEMS_BAY     3      mse               NaN  
             6      mse               NaN  
             12     mse               NaN  
PeMS_D7      3      mse               NaN  
             6      mse               NaN  
             12     mse               NaN  
SolarEnergy  1      mse          0.043829  
Weather      1      mse          0.144806

In [23]:
all_df

,,,
dataset,horizon,metric,model_type


In [214]:
# multi_df = all_df.unstack(2).loc[:,0]
# df_pivot = multi_df.pivot_table(index=['model_type', 'dataset'], columns='metric')
# df_pivot

horizon                         3                             6             \
metric                         mae      mape       mse       mae      mape   
model_type      dataset                                                      
BiSTGNNv6       METR_LA   0.190713  1.431686  0.168742  0.248017  1.505344   
                PEMS_BAY  0.166297  1.332880  0.118757  0.203388  1.569010   
                PeMS_D7   0.171326  1.084222  0.095415  0.215459  1.320459   
DCRNN           METR_LA   0.417376  1.784012  0.576841  0.441468  1.858663   
                PEMS_BAY  0.301349  1.623851  0.501967  0.322113  1.836862   
                PeMS_D7   0.337593  1.320793  0.339476  0.361160  1.483423   
GMAN            METR_LA   0.213674  1.509344  0.186236  0.266624  1.546768   
                PEMS_BAY  0.184999  1.399876  0.142327  0.223788  1.842992   
                PeMS_D7   0.183443  1.150248  0.106178  0.250846  1.588436   
GraphWavenet    METR_LA   0.180487  1.441376  0.167706  0.241237  1.521413   
                PEMS_BAY  0.163995  1.292594  0.118147  0.203279  1.596305   
                PeMS_D7   0.171262  1.058553  0.099486  0.222080  1.346786   
MTGNN           METR_LA   0.193838  1.389468  0.166066  0.241522  1.488167   
                PEMS_BAY  0.165529  1.369347  0.119063  0.202712  1.610928   
                PeMS_D7   0.168609  1.085353  0.092258  0.212067  1.356382   
STGCN           METR_LA   0.245755  1.582215  0.220596  0.286157  1.719533   
                PEMS_BAY  0.228808  2.076844  0.180041  0.261869  2.215809   
                PeMS_D7   0.368438  2.131399  0.364160  0.370833  2.120795   
fully_connected PeMS_D7   0.170527  1.050811  0.094468  0.214580  1.307827   

horizon                                   12                            24  \
metric                         mse       mae      mape       mse       mae   
model_type      dataset                                                      
BiSTGNNv6       METR_LA   0.239999  0.334581  1.587165  0.364865       NaN   
                PEMS_BAY  0.178997  0.242301  1.866390  0.260608       NaN   
                PeMS_D7   0.149284  0.262303  1.576395  0.219403       NaN   
DCRNN           METR_LA   0.612108  0.481197  1.858706  0.672288  0.535330   
                PEMS_BAY  0.522289  0.348372  2.126877  0.552800  0.383962   
                PeMS_D7   0.373024  0.401327  1.721107  0.433990  0.470910   
GMAN            METR_LA   0.262530  0.337828  1.671203  0.374530       NaN   
                PEMS_BAY  0.208677  0.282388  2.340087  0.321447       NaN   
                PeMS_D7   0.196453  0.305642  1.933865  0.280185       NaN   
GraphWavenet    METR_LA   0.238403  0.309127  1.567848  0.350884       NaN   
                PEMS_BAY  0.184156  0.256873  2.124807  0.277684       NaN   
                PeMS_D7   0.168872  0.294195  1.727582  0.280312       NaN   
MTGNN           METR_LA   0.233748  0.312503  1.624212  0.346836       NaN   
                PEMS_BAY  0.178550  0.241964  1.956990  0.254264       NaN   
                PeMS_D7   0.148266  0.266012  1.677156  0.225203       NaN   
STGCN           METR_LA   0.297554  0.351674  1.852275  0.403349  0.424660   
                PEMS_BAY  0.238756  0.300800  2.432629  0.313860  0.351139   
                PeMS_D7   0.367617  0.376783  2.183046  0.376894       NaN   
fully_connected PeMS_D7   0.147279  0.263990  1.592294  0.220662       NaN   

horizon                                       
metric                        mape       mse  
model_type      dataset                       
BiSTGNNv6       METR_LA        NaN       NaN  
                PEMS_BAY       NaN       NaN  
                PeMS_D7        NaN       NaN  
DCRNN           METR_LA   1.786363  0.760796  
                PEMS_BAY  2.373516  0.597106  
                PeMS_D7   1.907657  0.543679  
GMAN            METR_LA        NaN       NaN  
                PEMS_BAY       NaN       NaN  
                PeMS_D7        NaN       NaN  
Gr

# 数据处理


## .2 数据提取与小数点保留

1. 提取mse 数据
2. 数据更改索引方式，改为论文中方式
3. 小数点保留4位

In [253]:
df_pivot

model_type       BiSTGNNv6                     fully_connected            \
metric                 mae      mape       mse             mae      mape   
dataset  horizon                                                           
METR_LA  3        0.190713  1.431686  0.168742        0.194438  1.433156   
         6        0.248017  1.505344  0.239999        0.248848  1.507291   
         12       0.334581  1.587165  0.364865        0.328640  1.620991   
PEMS_BAY 3        0.166297  1.332880  0.118757             NaN       NaN   
         6        0.203388  1.569010  0.178997             NaN       NaN   
         12       0.242301  1.866390  0.260608        0.242262  1.844718   
PeMS_D7  3        0.171326  1.084222  0.095415        0.170527  1.050811   
         6        0.215459  1.320459  0.149284        0.214580  1.307827   
         12       0.262303  1.576395  0.219403        0.263990  1.592294   

model_type                  
metric                 mse  
dataset  horizon            
METR_LA  3        0.169902  
         6        0.241359  
         12       0.366621  
PEMS_BAY 3             NaN  
         6             NaN  
         12       0.260298  
PeMS_D7  3        0.094468  
         6        0.147279  
         12       0.220662

In [256]:
# .loc[:,0] 去掉第一列索引 （0）


# # result_df.to_excel('results.xlsx')
df_pivot = result_df.pivot_table(index=['dataset', 'horizon'], columns='metric')
# df_pivot


output = df_pivot.loc[:, (slice(None), ['mae', 'mse'])].round(4)
# output = df_pivot.loc[:, (slice(None), ['mae','mape', 'mse'])].round(4)
output = output.applymap(lambda x: f'{x:.4f}')
output


model_type       BiSTGNNv6            fully_connected            hetero_only  \
metric                 mae        mse             mae        mse         mae   
dataset  horizon                                                               
METR_LA  3          0.1907     0.1687          0.1944     0.1699      3.5096   
         6          0.2480     0.2400          0.2488     0.2414      4.2354   
         12         0.3346     0.3649          0.3286     0.3666      5.6452   
PEMS03   3         14.1035   519.0425         14.2359   522.3656     14.9291   
         6         15.3990   632.3217         15.3326   618.8236     15.6840   
         12            nan        nan         16.9941   811.0266     20.4830   
PEMS04   3         18.2607   849.3810         18.2135   845.9928     18.3477   
         6         19.2792   932.5225         19.1822   929.2974     19.2364   
         12        20.4452  1051.6983         20.5580  1057.0628     20.8149   
PEMS07   3         19.2383   917.8899         19.1481   914.9430     19.4490   
         6         20.8722  1070.5369         20.8412  1069.0638     20.8435   
         12        23.0703  1305.1304         23.0995  1295.1978     22.5053   
PEMS08   3         14.1308   471.6300         14.0341   465.6768     14.0432   
         6         15.2123   544.5602         15.1533   542.9227     15.3077   
         12        16.6159   657.5450         16.6978   660.0639     16.6941   
PEMS_BAY 3          0.1663     0.1188          1.2389     6.4863      1.2423   
         6          0.2034     0.1790          0.2022     0.1798      1.5388   
         12         0.2423     0.2606          0.2423     0.2603      1.9287   
PeMS_D7  3          0.1713     0.0954          0.1705     0.0945      1.8475   
         6          0.2155     0.1493          0.2146     0.1473      2.3840   
         12         0.2623     0.2194          0.2640     0.2207      2.9902   

model_type                  homo_only             
metric                  mse       mae        mse  
dataset  horizon                                  
METR_LA  3          61.3110    3.4885    61.1759  
         6          86.6443    4.7562    97.6230  
         12        132.8891    5.9132   134.4315  
PEMS03   3         543.9492   14.3115   524.1847  
         6         623.9797   15.6125   616.5187  
         12       1482.0208       nan        nan  
PEMS04   3         850.5627   18.2340   844.4482  
         6         926.7184   19.2020   933.2598  
         12       1068.6323   20.5457  1058.7480  
PEMS07   3         935.2945       nan        nan  
         6        1063.4507       nan        nan  
         12       1254.6148       nan        nan  
PEMS08   3         467.1726   14.0716   467.4702  
         6         554.7147   15.0918   538.6324  
         12        663.4585   16.8173   668.5488  
PEMS_BAY 3           6.6073    1.2430     6.5395  
         6          10.7412       nan        nan  
         12         16.5537       nan        nan  
PeMS_D7  3          10.7154    1.8587    10.7783  
         6          18.5652    2.3702    18.3768  
         12         29.7288    2.9847    29.3506

model_type              BiSTGNNv6  fully_connected  hetero_only  homo_only
dataset horizon metric                                                    
METR_LA 3       mae      3.548780         3.579419     3.509607   3.488463
                mape     0.077590         0.078440     0.077224   0.077288
                mse      0.168742         0.169902    61.311044  61.175907
                rmse     7.824927         7.897504     7.830102   7.821467
        6       mae      4.597614              NaN     4.235427   4.756249
...                           ...              ...          ...        ...
PeMS_D7 6       rmse     4.295256         4.303672     4.308558   4.286740
        12      mae      2.956619         3.003128     2.990238   2.984709
                mape     0.071834         0.072616     0.073839   0.072359
                mse      0.219403         0.220662    29.728799  29.350558
                rmse     5.362164         5.386747     5.452259   5.417358

[84 rows x 4 columns]

## .3 导出并且第一黑体、第二下划线

### +mape

In [6]:


# # result_df.to_excel('results.xlsx')
df_pivot = result_df.pivot_table(index=['dataset', 'horizon'], columns='metric')
# df_pivot


output = df_pivot.loc[:, (slice(None), ['mae','mape','rmse'])].round(4)
# output = df_pivot.loc[:, (slice(None), ['mae','mape', 'mse'])].round(4)
output = output.applymap(lambda x: f'{x:.4f}')
output


model_type       BiSTGNNv6                   fully_connected                   \
metric                 mae     mape     rmse             mae    mape     rmse   
dataset  horizon                                                                
METR_LA  3          3.5488   0.0776   7.8249          3.5794  0.0784   7.8975   
         6          4.5976   0.0961   9.5308             nan     nan      nan   
         12         5.9761   0.1181  11.6762          5.8771  0.1177  11.6117   
PEMS03   3         14.1035   0.1700  22.7805         14.2359  0.1665  22.8532   
         6         15.3990   0.1798  25.1408         15.3326  0.1778  24.8742   
         12         0.0140  -0.5149   0.0205         16.9941  0.1955  28.4709   
PEMS04   3         18.2607   0.1231  29.1440         18.2135  0.1240  29.0859   
         6         19.2792   0.1329  30.5372         19.1822  0.1311  30.4843   
         12        20.4452   0.1399  32.4294         20.5580  0.1414  32.5120   
PEMS07   3         19.2383   0.0826  30.2967         19.1481  0.0816  30.2478   
         6         20.8722   0.0890  32.7185         20.8412  0.0905  32.6952   
         12        23.0703   0.1012  36.1256         23.0995  0.1013  35.9887   
PEMS08   3         14.1308   0.0935  21.7166         14.0341  0.0931  21.5789   
         6         15.2123   0.1017  23.3356         15.1533  0.1003  23.3003   
         12        16.6159   0.1114  25.6418         16.6978  0.1110  25.6907   
PEMS_BAY 3          1.2454   0.0260   2.5562          1.2389  0.0260   2.5468   
         6          1.5615   0.0343   3.2869          1.5553  0.0344   3.2731   
         12         1.9202   0.0445   4.0770          1.9115  0.0444   4.0720   
PeMS_D7  3          0.1759   0.0199   0.3081          1.8544  0.0428   3.2828   
         6          2.3813   0.0564   4.2953          2.3804  0.0566   4.3037   
         12         2.9566   0.0718   5.3622          3.0031  0.0726   5.3867   

model_type       hetero_only                  homo_only                   
metric                   mae    mape     rmse       mae    mape     rmse  
dataset  horizon                                                          
METR_LA  3            3.5096  0.0772   7.8301    3.4885  0.0773   7.8215  
         6            4.2354  0.0915   9.3082    4.7562  0.1003   9.8663  
         12           5.6452  0.1155  11.5269    5.9132  0.1189  11.5926  
PEMS03   3           14.9291  0.2034  23.2994   14.3115  0.1727  22.8935  
         6           15.6840  0.1996  24.9771   15.6125  0.1864  24.8287  
         12          20.4830  0.2482  36.5628   17.2131  0.2118  28.4225  
PEMS04   3           18.3477  0.1247  29.1624   18.2340  0.1251  29.0590  
         6           19.2364  0.1311  30.4418   19.2020  0.1309  30.5491  
         12          20.8149  0.1470  32.6780   20.5457  0.1405  32.5375  
PEMS07   3           19.4490  0.0828  30.5748   19.2719  0.0816  30.3568  
         6           20.8435  0.0915  32.6101       nan     nan      nan  
         12          22.5053  0.0994  35.4195       nan     nan      nan  
PEMS08   3           14.0432  0.0930  21.6131   14.0716  0.0932  21.6205  
         6           15.3077  0.1030  23.5425   15.0918  0.0990  23.2081  
         12          16.6941  0.1122  25.7435   16.8173  0.1121  25.8510  
PEMS_BAY 3            1.2423  0.0261   2.5704    1.2430  0.0262   2.5572  
         6            1.5388  0.0341   3.2773    1.5619  0.0346   3.2934  
         12           1.9287  0.0446   4.0676    1.9181  0.0441   4.0904  
PeMS_D7  3            1.8475  0.0428   3.2734    1.8587  0.0430   3.2830  
         6            2.3840  0.0570   4.3086    2.3702  0.0561   4.2867  
         12           2.9902  0.0738   5.4523    2.9847  0.0724   5.4174

In [7]:
# 假设 df 是你的 DataFrame
# 创建一个 ExcelWriter 对象

def style_rule(best, second_best):
    if best:
        return 'font-weight: bold'
    elif second_best:
        return 'text-decoration: underline'
    else:
        return ''
with pd.ExcelWriter('./output.xlsx') as writer:
    # 定义一个函数，用于找到最好和第二好的结果，并将它们设置为黑体
    def highlight_best(s):
        # 分离 mae 和 mse
        # print(s)
        # 分离 mae 和 mse
        mae = s.xs('mae', level='metric')
        # mse = s.xs('mse', level='metric')
        rmse = s.xs('rmse', level='metric')
        mape = s.xs('mape', level='metric')
        
        # 找到 mae 和 mse 的最好和第二好的结果
        is_best_mae = mae == mae.min()
        is_second_best_mae = mae.rank(method='min') == 2
        
        is_best_rmse = rmse == rmse.min()
        is_second_best_rmse = rmse.rank(method='min') == 2

#         is_best_mse = mse == mse.min()
#         is_second_best_mse = mse.rank(method='min') == 2
        
        is_best_mape = mape == mape.min()
        is_second_best_mape = mape.rank(method='min') == 2

        # is_best = is_best_mse | is_best_mae
        if s.name[0] == 'Weather' and s.name[1] ==3:
            print(is_best_mae)
        if s.name[0] == 'Weather' and s.name[1] ==3:
            print(is_best_mse)
        # 创建一个新的序列来存储最终的样式
        



        # 交错合并
        is_best_combined = [val for pair in zip(is_best_mae,is_best_mape,is_best_rmse) for val in pair]
        is_second_best_combined = [val for pair in zip(is_second_best_mae,is_second_best_mape,is_second_best_rmse) for val in pair]
        style =  [style_rule(best,second_best) for best, second_best in zip(is_best_combined,is_second_best_combined)  ]
        
        print(style)
        return style
        # return ['color: black; font-weight: bold' if v else 'color: darkgreen;  font-weight: bold' if sb else '' for v, sb in zip(is_best, is_second_best)]
        # return ['color: black; font-weight: bold' if v else 'color: darkgreen;  font-weight: bold' if sb else '' for v, sb in zip(is_best, is_second_best)]

    # 对 DataFrame 的每一行应用 highlight_best 函数
    styled_df = output.style.apply(highlight_best, axis=1)

    # 将样式化的 DataFrame 写入 Excel 文件
    styled_df.to_excel(writer, engine='openpyxl')


['', '', 'text-decoration: underline', '', '', '', 'text-decoration: underline', 'font-weight: bold', '', 'font-weight: bold', 'text-decoration: underline', 'font-weight: bold']
['text-decoration: underline', 'text-decoration: underline', 'text-decoration: underline', '', '', '', 'font-weight: bold', 'font-weight: bold', 'font-weight: bold', '', '', '']
['', '', '', 'text-decoration: underline', 'text-decoration: underline', '', 'font-weight: bold', 'font-weight: bold', 'font-weight: bold', '', '', 'text-decoration: underline']
['font-weight: bold', 'text-decoration: underline', 'font-weight: bold', 'text-decoration: underline', 'font-weight: bold', 'text-decoration: underline', '', '', '', '', '', '']
['text-decoration: underline', 'text-decoration: underline', '', 'font-weight: bold', 'font-weight: bold', 'text-decoration: underline', '', '', '', '', '', 'font-weight: bold']
['font-weight: bold', 'font-weight: bold', 'font-weight: bold', 'text-decoration: underline', 'text-decoration

### mae+mse

In [241]:
# .loc[:,0] 去掉第一列索引 （0）


# # result_df.to_excel('results.xlsx')
df_pivot = result_df.pivot_table(index=['dataset', 'horizon'], columns='metric')
# df_pivot


output = df_pivot.loc[:, (slice(None), ['mae', 'mse'])].round(4)
# output = df_pivot.loc[:, (slice(None), ['mae','mape', 'mse'])].round(4)
output = output.applymap(lambda x: f'{x:.4f}')
output


model_type           BiSTGNNv6         TNTCN_TCNONLY         fully_connected  \
metric                     mae     mse           mae     mse             mae   
dataset      horizon                                                           
ETTh1        3          0.3120  0.2104        0.3149  0.2129          0.3131   
             6          0.3593  0.2699        0.3664  0.2794          0.3597   
             12         0.3814  0.2968        0.3861  0.3035          0.3821   
             24         0.3900  0.3111        0.3922  0.3127          0.3942   
ETTh2        3          0.1567  0.0467        0.1569  0.0468          0.1571   
             6          0.1829  0.0644        0.1821  0.0642          0.1819   
             12         0.1992  0.0758        0.1977  0.0754          0.1983   
             24         0.2106  0.0837        0.2130  0.0864          0.2109   
ETTm1        3          0.2148  0.1137        0.2151  0.1140          0.2149   
             6          0.2592  0.1547        0.2586  0.1553          0.2584   
             12         0.3123  0.2121        0.3133  0.2131          0.3122   
             24         0.3626  0.2758        0.3623  0.2767          0.3624   
ETTm2        3          0.1158  0.0269        0.1181  0.0276          0.1145   
             6          0.1353  0.0359        0.1376  0.0368          0.1354   
             12         0.1567  0.0474        0.1598  0.0495          0.1568   
             24         0.1836  0.0655        0.1847  0.0662          0.1837   
ExchangeRate 3          0.0464  0.0042        0.0462  0.0042          0.0463   
             6          0.0652  0.0079        0.0654  0.0079          0.0652   
             12         0.0922  0.0154        0.0906  0.0149          0.0922   
             24         0.1255  0.0272        0.1319  0.0296          0.1256   
Weather      3          0.0793  0.0662        0.0805  0.0683          0.0795   
             6          0.0986  0.0843        0.1014  0.0855          0.0977   
             12         0.1319  0.1080        0.1313  0.1084          0.1326   
             24         0.1923  0.1439        0.1715  0.1358          0.1972   

model_type                   hetero_only         homo_only          
metric                   mse         mae     mse       mae     mse  
dataset      horizon                                                
ETTh1        3        0.2117      0.3120  0.2097       nan     nan  
             6        0.2703      0.3586  0.2686       nan     nan  
             12       0.2980      0.3835  0.2985    0.3811  0.2964  
             24       0.3152      0.3892  0.3125       nan     nan  
ETTh2        3        0.0468      0.1570  0.0467    0.1574  0.0469  
             6        0.0639      0.1834  0.0647    0.1821  0.0639  
             12       0.0755      0.1998  0.0761    0.1996  0.0758  
             24       0.0837      0.2150  0.0862    0.2172  0.0876  
ETTm1        3        0.1138      0.2152  0.1140    0.2153  0.1138  
             6        0.1542      0.2601  0.1540    0.2593  0.1528  
             12       0.2122      0.3101  0.2096    0.3112  0.2093  
             24       0.2763      0.3642  0.2736    0.3618  0.2712  
ETTm2        3        0.0266      0.1155  0.0270    0.1157  0.0269  
             6        0.0359      0.1358  0.0361    0.1361  0.0362  
             12       0.0474      0.1562  0.0472    0.1569  0.0477  
             24       0.0655      0.1841  0.0656    0.1842  0.0660  
ExchangeRate 3        0.0042      0.0464  0.0042    0.0464  0.0042  
             6        0.0079      0.0654  0.0079    0.0659  0.0081  
             12       0.0154      0.0914  0.0152    0.0915  0.0152  
             24       0.0272      0.1263  0.0275    0.1259  0.0274  
Weather      3        0.0662      0.0795  0.0662       nan     nan  
             6        0.0841      0.0981  0.0838       nan     nan  
             12       0.1079      0.1353  0.1084       nan     nan  
             24       0.1443      0.2084  0.1

In [242]:
# 假设 df 是你的 DataFrame
# 创建一个 ExcelWriter 对象

def style_rule(best, second_best):
    if best:
        return 'font-weight: bold'
    elif second_best:
        return 'text-decoration: underline'
    else:
        return ''
with pd.ExcelWriter('./output.xlsx') as writer:
    # 定义一个函数，用于找到最好和第二好的结果，并将它们设置为黑体
    def highlight_best(s):
        # 分离 mae 和 mse
        # print(s)
        # 分离 mae 和 mse
        mae = s.xs('mae', level='metric')
        mse = s.xs('mse', level='metric')
        
        # 找到 mae 和 mse 的最好和第二好的结果
        is_best_mae = mae == mae.min()
        is_second_best_mae = mae.rank(method='min') == 2
        is_best_mse = mse == mse.min()
        is_second_best_mse = mse.rank(method='min') == 2
        # is_best = is_best_mse | is_best_mae
        if s.name[0] == 'Weather' and s.name[1] ==3:
            print(is_best_mae)
        if s.name[0] == 'Weather' and s.name[1] ==3:
            print(is_best_mse)
        # 创建一个新的序列来存储最终的样式
        



        # 交错合并
        is_best_combined = [val for pair in zip(is_best_mae, is_best_mse) for val in pair]
        is_second_best_combined = [val for pair in zip(is_second_best_mae, is_second_best_mse) for val in pair]
        style =  [style_rule(best,second_best) for best, second_best in zip(is_best_combined,is_second_best_combined)  ]
        
        print(style)
        return style
        # return ['color: black; font-weight: bold' if v else 'color: darkgreen;  font-weight: bold' if sb else '' for v, sb in zip(is_best, is_second_best)]
        # return ['color: black; font-weight: bold' if v else 'color: darkgreen;  font-weight: bold' if sb else '' for v, sb in zip(is_best, is_second_best)]

    # 对 DataFrame 的每一行应用 highlight_best 函数
    styled_df = output.style.apply(highlight_best, axis=1)

    # 将样式化的 DataFrame 写入 Excel 文件
    styled_df.to_excel(writer, engine='openpyxl')


['font-weight: bold', 'text-decoration: underline', '', '', '', '', 'font-weight: bold', 'font-weight: bold', '', '']
['text-decoration: underline', 'text-decoration: underline', '', '', '', '', 'font-weight: bold', 'font-weight: bold', '', '']
['text-decoration: underline', 'text-decoration: underline', '', '', '', '', '', '', 'font-weight: bold', 'font-weight: bold']
['text-decoration: underline', 'font-weight: bold', '', '', '', '', 'font-weight: bold', 'text-decoration: underline', '', '']
['font-weight: bold', 'font-weight: bold', 'text-decoration: underline', '', '', '', '', 'font-weight: bold', '', '']
['', '', 'text-decoration: underline', '', 'font-weight: bold', 'font-weight: bold', '', '', 'text-decoration: underline', 'font-weight: bold']
['', '', 'font-weight: bold', 'font-weight: bold', 'text-decoration: underline', 'text-decoration: underline', '', '', '', '']
['font-weight: bold', 'font-weight: bold', '', '', 'text-decoration: underline', 'font-weight: bold', '', '', ''